# Pymaceuticals Inc.
---

### Analysis

- While only 4 drug regimens were deep dived, of the 4, Infubinol was the only one with an outlier. This seems to indicate that the other 3 produce more consistent results. However, the existence of that one outlier may indicate that there is something unique about that drug regimen, or the group it was tested on. 

- As evidenced by the box plot, the drug regimens all generally started at very different tumor volumes. In further statistical analysis, I would create a similar dataframe as the one that listed all the ending tumor volumes, with starting volumes. This should help identify other important factors of the drugs’ efficacies, including time to response, and would allow us to conduct further stastical tests, such as covariance analysis.

- Regarding the topic of time to respond, it seems apparent from the line plot of the effects of Capomulin on test subject l509, that Capomulin has about a 20 day time to respond. Once it starts responding however, it significantly reduces tumor volume until about day 35m, at which point it starts to rise again. 
 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
mouse_df = pd.merge(study_results, mouse_metadata, on="Mouse ID", how='left')

# Display the data table for preview
mouse_df.head()

In [ ]:
# Checking the number of mice.
print(len(mouse_df['Mouse ID'].unique()))

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
dupes = mouse_df[mouse_df.duplicated(subset=['Mouse ID', 'Timepoint'], keep=False)]
unique_dupes = dupes['Mouse ID'].unique()
unique_dupes_array = np.array(unique_dupes)
print(repr(unique_dupes_array))



In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
dupe_data = mouse_df[mouse_df["Mouse ID"] == 'g989']
dupe_data

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
duplicate_mouse_ids = mouse_df[mouse_df.duplicated(subset=['Mouse ID', 'Timepoint'], keep=False)]['Mouse ID'].unique()
clean_mouse_df = mouse_df[~mouse_df['Mouse ID'].isin(duplicate_mouse_ids)]
clean_mouse_df.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
print(len(clean_mouse_df['Mouse ID'].unique()))

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.
mean_drug_regimen = clean_mouse_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
med_drug_regimen = clean_mouse_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
var_drug_regimen = clean_mouse_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
std_drug_regimen = clean_mouse_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
sem_drug_regimen = clean_mouse_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()

# put summary statistics into new dataframe
summary_stats = pd.DataFrame({
    "Mean Tumor Volume": mean_drug_regimen,
    "Median Tumor Volume": med_drug_regimen,
    "Tumor Volume Variance": var_drug_regimen,
    "Tumor Volume Std. Dev": std_drug_regimen,
    "Tumor Volume Std. Err": sem_drug_regimen
})
summary_stats

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.

drug_count = clean_mouse_df['Drug Regimen'].value_counts()
drug_count.plot(kind='bar', figsize=(7,5))
plt.ylabel("# of Observed Mouse Timestamps")
plt.show()

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.

plt.bar(drug_count.index, drug_count.values)
plt.xlabel("Drug Regimen")
plt.ylabel("# of Observed Mouse Timestamps")
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas

gender_count = clean_mouse_df['Sex'].value_counts()
gender_count.plot(kind='pie', figsize=(7,5), autopct='%1.1f%%')
plt.ylabel("Sex")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

plt.pie(gender_count, labels=gender_count.index, autopct='%1.1f%%')
plt.ylabel("Sex")
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
last_timepoint = clean_mouse_df.groupby('Mouse ID')['Timepoint'].max().reset_index()
last_timepoint = last_timepoint.reset_index()

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
final_mouse_df = pd.merge(last_timepoint, clean_mouse_df, on=['Mouse ID', 'Timepoint'], how='left')


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatments = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']

# Create empty list to fill with tumor vol data (for plotting)
final_tumor_volumes = {drug: [] for drug in treatments}

for treatment in treatments:
    final_volumes = final_mouse_df[final_mouse_df['Drug Regimen'] == treatment]['Tumor Volume (mm3)']
    # Append these volumes to the corresponding list in the dictionary
    final_tumor_volumes[treatment].extend(final_volumes)

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatments:
    volumes = pd.Series(final_tumor_volumes[drug])

    quartiles = volumes.quantile([.25, .5, .75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq

    lower_bound = lowerq - (1.5 * iqr)
    upper_bound = upperq + (1.5 * iqr)
    
    # Determine outliers using upper and lower bounds
    outliers = volumes[(volumes < lower_bound) | (volumes > upper_bound)]
    print(f"Potential outliers for {drug}: {outliers}\n")


In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.
treatments = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']
final_df_filtered = final_mouse_df[final_mouse_df['Drug Regimen'].isin(treatments)]
final_df_filtered = final_df_filtered.reset_index()
final_df_filtered.boxplot(by='Drug Regimen', column='Tumor Volume (mm3)', grid=False)
plt.ylabel('Final Tumor Volume (mm3)')
plt.show()



## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
single_rat_filtered = clean_mouse_df[clean_mouse_df['Mouse ID'] == 'l509']
single_rat_filtered.plot(kind='line', x='Timepoint', y='Tumor Volume (mm3)')
plt.xlabel("Timepoint (days)")
plt.ylabel("Tumor Volume (mm3)")
plt.title("Capomulin treatment of mouse l509")
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen

# filter for only Capomulin drug regimen
capo_filtered = clean_mouse_df[clean_mouse_df['Drug Regimen'] == 'Capomulin']

# get averages of tumor volumes for time points
capo_avg = capo_filtered.groupby('Mouse ID').agg({'Tumor Volume (mm3)': 'mean', "Weight (g)": 'mean'})
capo_avg = capo_avg.reset_index()

# plot scatter plot and set labels
plt.scatter(capo_avg['Weight (g)'], capo_avg['Tumor Volume (mm3)'])
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen

# get correlation factors in their own variables and then calculate it
weight = capo_avg.iloc[:,2]
volume = capo_avg.iloc[:,1]
correlation = st.pearsonr(weight, volume)
print(f"The correlation between mouse weight and the average tumor volume is {round(correlation[0],2)}")

# calculate linear regression and add i to the scatter plot
v_slope, v_int, v_r, v_p, v_std_err = st.linregress(weight, volume)
v_fit = v_slope * weight + v_int
plt.scatter(capo_avg['Weight (g)'], capo_avg['Tumor Volume (mm3)'])
plt.plot(weight, v_fit,"--", color='red')
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.show()
